<a href="https://colab.research.google.com/github/kacper-dobrow/tensorflow-course/blob/main/07_milestone_project_1_food_vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Milestone project 1: Food Vision Big

## Check GPU

Not all GPUs are compatible with mixed  precision training. Tesla T4 is compatible.

https://developer.nvidia.com/cuda-gpus

In [ ]:
!nvidia-smi

## Get helper functions

In [ ]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

In [ ]:
from helper_functions import plot_loss_curves, compare_historys, make_confusion_matrix, create_tensorboard_callback

## Use TensorFlow Datasets to download data

https://www.tensorflow.org/datasets/overview?hl=pl


In [ ]:
# Get tensorflow datasets
import tensorflow_datasets as tfds

In [ ]:
# List all available datasets
datasets_list = tfds.list_builders()
print("food101" in datasets_list)

In [ ]:
# Download data already in tensor format
(train_data, test_data), ds_info = tfds.load(name="food101",
                                             split=["train","validation"],
                                             shuffle_files=True,
                                             as_supervised=True,
                                             with_info=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

In [ ]:
# Features of the dataset
ds_info.features

In [ ]:
# Get class names
class_names = ds_info.features["label"].names

In [ ]:
# Take one sample of the train data
train_one_sample = train_data.take(1) # samples are in format (image tensor, label)

In [ ]:
# What does one sample of our training data look like?
train_one_sample

In [ ]:
# Output info about our training sample
for image, label in train_one_sample:
  print(f"""
  Image shape: {image.shape}
  Image datatype: {image.dtype}
  Target class from Food101 (tensor form): {label}
  Class name (str form): {class_names[label.numpy()]}
  """)

In [ ]:
# What does the image tensor from TFDS look like?
image

In [ ]:
# What are the min and max values of our image tensor?
import tensorflow as tf
tf.reduce_min(image), tf.reduce_max(image)

### Plot an image from the dataset


In [ ]:
import matplotlib.pyplot as plt
plt.imshow(image)
plt.axis(False)
plt.title(class_names[label.numpy()]);

## Create preprocessing functions for our data

What we know ablut our data:
* In `uint8` datatype
* Many different sizes
* Not scaled - pixel values between 0 and 255

The models like:
* Data in `float32` datatype or for mixed precision also `float16`
* For batches, TensorFlow likes all the tensors within a batch to be the same size
* Scaled/normalized tensors generally perform better


In [ ]:
# Make a function for preprocessing images
def preprocess_img(image, label, img_shape=224):
  """
  Converts image datatype from uint8 to float32 and reshapes image to (img_shape, img_shape, color_channels)
  """
  image = tf.image.resize(image, [img_shape, img_shape]) # why list?
  return tf.cast(image, tf.float32), label

In [ ]:
# Preprocess a simple sample image and preprocess it
preprocessed_img = preprocess_img(image, label)[0]